# Tensorflow のスクリプトに対してハイパーパラメータのチューニングを行う

#### ノートブックに含まれる内容

- Tensorflow のスクリプトに対して，ハイパーパラーメータのチューニングを行う方法
- ハイパーパラーメータチューニングの概要説明と基本的な使い方

#### ノートブックで使われている手法の詳細

- アルゴリズム: CNN
- データ: MNIST

## セットアップ

必要なパラメタをセットアップします．


In [ ]:
import boto3
from time import gmtime, strftime
import sagemaker

role = sagemaker.get_execution_role()

以下を実行する前に，**<span style="color: red;">`sagemaker/hpo-tensorflow-high/XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/hpo-tensorflow-high/XX'

## データのロード

Tensorflow 経由で MNIST データをロードします．その上で，学習用，検証用，テスト用の 3 つにデータを分割します．

In [ ]:
import utils
from tensorflow.contrib.learn.python.learn.datasets import mnist
import tensorflow as tf

data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

データの準備が終わったら，`sagemaker.Session().upload_data()` を使ってデータを S3 にアップロードします．

In [ ]:
inputs = sagemaker.Session().upload_data(path='data', bucket=bucket, key_prefix=prefix+'/data/mnist')
print (inputs)

## ハイパーパラメータチューニングジョブの実行

続いて，チューニングジョブのセットアップを行い実行します．ここには以下の 4 つの処理が含まれます．

1. 通常の学習ジョブのときと同様に，Tensorflow クラスのオブジェクトを作成します
1. チューニングしたいハイパーパラメータ名と範囲を，Dictionary 型で指定します
1. チューニングの評価を行うためのターゲットメトリクスを指定します
1. 実際にチューニングジョブを実行します

### 1. Tensorflow オブジェクトの作成

これは，通常の学習ジョブを実行するとの全く同じ手順です．

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='mnist.py',
                  role=role,
                  training_steps=100, 
                  evaluation_steps=10,
                  train_instance_count=1,
                  train_instance_type='ml.m4.xlarge')

### 2. ハイパーパラメータのリストを作成

次に，チューニングしたいハイパーパラメータのリストを作成します．ハイパーパラメータの中身に応じたオブジェクトがあるので，これを使用します．ハイパーパラメータがカテゴリの場合は探索対象のカテゴリのリストを，連続値の場合は範囲を指定する形にしてください．なお整数の場合は，通常の連続値とは異なるオブジェクトを用いて指定します．

- カテゴリ: `CategoricalParameter(list)`
- 連続値: `ContinuousParameter(min, max)`
- 整数: `IntegerParameter(min, max)`

今回は，`learning_rate` をターゲットとして選択しました．

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {'learning_rate': ContinuousParameter(0.01, 0.2)}

### 3. ターゲットメトリクスの指定

続いて，チューニングの評価をするためのメトリクスを指定します．このメトリクスは，Sagemaker 側でジョブ実行時の標準出力から正規表現で抽出します．対象となるメトリクスが標準出力ログに出力されるように，自身のスクリプトを記述してください．ここでいう標準出力ログとは，ジョブ実行時に Cloud Watch Logs に出力されるログのことを指します．

このターゲットメトリクスを最小化するか，最大化するかを選択することができます．デフォルトは最大化となります．

ここでは，損失関数の値をターゲットとして，これを最小化することを目指します．

In [ ]:
objective_metric_name = 'loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'loss',
                       'Regex': 'loss = ([0-9\\.]+)'}]

### 4. チューニングジョブの実行

以上の準備が終わったら，チューニングジョブのオプビェクトを作成して，`fit()` で実際に実行します．その際に **<span style="color: red;">`base_tuning_job_name` の `DEMO-hpo-tensorflow-XX` にある `XX` を指定された適切な数字に変更</span>**してください

`HyperparameterTuner` の詳細については[ドキュメント](https://sagemaker.readthedocs.io/en/latest/tuner.html)をご確認ください．


In [ ]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=9,
                            max_parallel_jobs=3,
                            objective_type=objective_type,
                            base_tuning_job_name='hpo-tensorflow-XX')

tuner.fit(inputs)

チューニングジョブの実行状況は，`boto3` クライアント経由で確認することが可能です．

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']